In [1]:
import pandas as pd
import numpy as np
import _pickle as cPickle
from datetime import timedelta
from tqdm import tqdm, tqdm_notebook

from sklearn.svm import SVC
import gc
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

C:\Users\zhav1k\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df1 = pd.read_csv('../custom_data/Feature_Engineering/for_cloud.csv')

In [3]:
df1.head()

,ticker,Date,Adj Close,Close,High,Low,Open,Volume,Name,Sector
0,AAL,2008-12-31,7.405228,7.73,7.87,7.48,7.48,4194100.0,American Airlines Group,Industrials
1,AAL,2009-01-02,8.037499,8.39,8.48,7.67,7.73,5167000.0,American Airlines Group,Industrials
2,AAL,2009-01-05,7.980019,8.33,8.39,7.96,8.38,3457100.0,American Airlines Group,Industrials
3,AAL,2009-01-06,8.679349,9.06,9.21,8.13,8.15,5731000.0,American Airlines Group,Industrials
4,AAL,2009-01-07,8.698509,9.08,9.47,8.66,8.66,5468900.0,American Airlines Group,Industrials


In [4]:
# df1 = pd.read_pickle('main_dataframe_without_ta.pkl')
# df2 = pd.read_pickle('main_dataframe_ta_features_only.pkl')
# df3 = pd.read_pickle('main_dataframe_sentiments_0302.pkl')

In [5]:
df1.isna().sum()

ticker           0
Date             0
Adj Close    52056
Close        52056
High         52056
Low          52056
Open         52056
Volume       52056
Name         44712
Sector       44712
dtype: int64

In [6]:
df1.dropna(axis = 0, how = 'any', inplace = True)
df1.isna().sum()

ticker       0
Date         0
Adj Close    0
Close        0
High         0
Low          0
Open         0
Volume       0
Name         0
Sector       0
dtype: int64

In [7]:
# df3.drop(['ticker'], axis = 1, inplace = True)
# df2.drop(['index', 'Adj Close', 'Close','High', 'Low', 'Open', 'Volume', 'Name', 'Sector'], axis = 1, inplace = True)
# df1 = pd.merge(df1, df3, left_index=True, right_index=True)
# df1 = pd.merge(df1, df2, how = 'left', on = ['ticker', 'Date'])
# print(df1.columns.get_loc('coord_0'), df1.columns.get_loc('coord_299'))

In [8]:
# with open('PCA_title.pkl', 'rb') as fid:
#     pca = cPickle.load(fid)

In [9]:
# colu = list(df1.iloc[:, 11:311].columns)
# title_pca = df1.iloc[:, 11:311]
# df1.drop(colu, axis = 1, inplace = True)
# final_title = pca.transform(title_pca)
# final_title = pd.DataFrame(final_title).add_prefix('coord_')
# df1 = pd.merge(df1, final_title, left_index=True, right_index=True)

In [10]:
df1['shifted_close'] = df1.groupby('ticker')['Adj Close'].shift(-1)
df1['tgt'] = (df1['shifted_close'] - df1['Adj Close']).div(df1['shifted_close']) * 100
df1['target'] = 0
df1.loc[(df1['tgt'] >= 0), 'target'] = 1
df1.loc[(df1['tgt'] < 0), 'target'] = 0
df1.drop(['Name', 'Sector', 'shifted_close', 'tgt'], axis = 1 , inplace = True)

In [11]:
df1['Date_dt'] = pd.to_datetime(df1['Date'], format='%Y%m%d', errors='ignore')
df1['year'] = pd.DatetimeIndex(df1['Date']).year
df1.drop(['Date'], axis = 1, inplace = True)

In [12]:
df1[df1['Date_dt'] > '2018-01-01'].head()

,ticker,Adj Close,Close,High,Low,Open,Volume,target,Date_dt,year
2266,AAL,52.482700,52.990002,53.099998,51.900002,52.330002,4084700.0,0,2018-01-02,2018
2267,AAL,51.838924,52.340000,52.860001,52.060001,52.860001,5074900.0,1,2018-01-03,2018
2268,AAL,52.165760,52.669998,54.400002,52.270000,52.480000,3557100.0,0,2018-01-04,2018
2269,AAL,52.145950,52.650002,52.840000,52.430000,52.779999,2967800.0,0,2018-01-05,2018
2270,AAL,51.630936,52.130001,52.639999,51.930000,52.599998,3515800.0,0,2018-01-08,2018


In [13]:
years = list(df1.year.unique())
burning_period = timedelta(days = 10)

In [14]:
fcol = [c for c in df1 if c not in ['Date_datetime', 'ticker', 'target', 'Date_dt', 'year']]

In [15]:
# for num_year in range(1, len(years)):
#     up_to = pd.to_datetime('2009-01-01') + timedelta(days = num_year*365)
#     train = df1[df1['Date_datetime'] <= up_to]
#     validation = df1[(df1['Date_datetime'] > (up_to + burning_period)) & (df1['Date_datetime'] <= up_to +timedelta(days = (num_year) * 365))]
    
    
#     #model should be here.

In [15]:
df1['target'].value_counts()

1    618562
0    549915
Name: target, dtype: int64

In [16]:
df1['Date_datetime'] = pd.to_datetime(df1['Date_dt'])

In [17]:
def evaluate_model_lgbm(df_train, target_train, df_val, target_val, params):
    model = LGBMClassifier(objective ='binary',
                           boosting ='gbdt', #dart
                           n_jobs = -1,
                           verbose_eval = -1,
                           learning_rate = 0.05,
                           **params)
    model.fit(df_train, target_train)
    return(model.score(df_train, target_train),accuracy_score(target_val, model.predict(df_val)))


param_dist_lgbm = {'num_leaves': [27, 31, 61, 81, 127, 197, 231, 275, 302],
              'bagging_fraction': [0.5, 0.7, 0.8, 0.9],
              'min_data_in_leaf': [100, 300, 500, 700, 1000, 1300, 1700, 2000, 3000],
              
              #'learning_rate': [0.001, 0.005, 0.01, 0.05, 0.1, 0.3, 0.5],
              'min_data': [300, 400, 450, 500, 550, 650],
              'max_bin': [5, 12, 18, 20, 22, 30],
              'bagging_freq': [3, 9, 11, 15, 17, 23, 31],
              'max_depth': [3, 4, 5, 6, 7, 9, 11],       
              'feature_fraction': [0.5, 0.7, 0.8, 0.9],
              'lambda_l1': [0, 0.1, 0.2, 0.3, 0.05, 0.005],
              'lambda_l2': [0, 0.1, 0.2, 0.3, 0.05, 0.005]
              
                } 






def evaluate_model_rf(df_train, target_train, df_val, target_val, params):
    model = RandomForestClassifier(
                           n_jobs = -1,
                           verbose = -1,
                           **params)
    model.fit(df_train, target_train)
    return(model.score(df_train, target_train),accuracy_score(target_val, model.predict(df_val)))


param_dist_rf = {'n_estimators': [100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750],
                   'max_features': ['auto', 'sqrt'],
# Maximum number of levels in tree
                    'max_depth': [3,4,5,6,7,8,9,10,11,12,13,14,15],
# Minimum number of samples required to split a node
                    'min_samples_split': [2, 5, 10, 15, 20, 25, 30, 35, 100, 150],
# Minimum number of samples required at each leaf node
                    'min_samples_leaf': [1, 2, 4, 8, 10, 12, 15, 20, 25, 35, 55, 100, 150],
# Method of selecting samples for training each tree
                    'bootstrap' : [True, False]
              
                }


def evaluate_model_logistic(df_train, target_train, df_val, target_val, params):
    model = LogisticRegression(penalty = 'l2', n_jobs = 1, **params)
    model.fit(df_train, target_train)
    return(model.score(df_train, target_train), accuracy_score(target_val, model.predict(df_val)))
    
    
    
param_dist_lr = {'C': [0.0001, 0.00001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
                'fit_intercept': [True, False]
                }

def evaluate_model_svm(df_train, target_train, df_val, target_val, params):
    model = SVC(kernel = 'rbf', **params)
    model.fit(df_train, target_train)
    return(model.score(df_train, target_train),accuracy_score(target_val, model.predict(df_val)))

param_dist_svc = {'C': [0.0001, 0.001, 0.01, 0.1, 0.15, 0.20, 0.5, 0.8, 1, 2, 3 ,4, -1, -2, -3, -0.1, -0.001],
              'gamma': [-3, -2.5, -2, -1.5, -1, -0.1, -0.001, 0.001, 0.1, 0.2, 1, 2, 3]}





In [62]:
score, params, lst_train, lst_test, all_params = hyperparameter_search(10,param_dist_lgbm, years, df1, burning_period, evaluate_model_lgbm )

  0%|                                                                                            | 0/9 [00:00<?, ?it/s]C:\Users\zhav1k\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zhav1k\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
 11%|█████████▎                                                                          | 1/9 [00:01<00:13,  1.65s/it]C:\Users\zhav1k\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an erro

 33%|████████████████████████████                                                        | 3/9 [00:13<00:26,  4.46s/it]C:\Users\zhav1k\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zhav1k\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
 44%|█████████████████████████████████████▎                                              | 4/9 [00:22<00:27,  5.54s/it]C:\Users\zhav1k\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an erro

 67%|████████████████████████████████████████████████████████                            | 6/9 [00:53<00:26,  8.87s/it]C:\Users\zhav1k\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zhav1k\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
 78%|█████████████████████████████████████████████████████████████████▎                  | 7/9 [01:14<00:21, 10.68s/it]C:\Users\zhav1k\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an erro

  0%|                                                                                            | 0/9 [00:00<?, ?it/s]C:\Users\zhav1k\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zhav1k\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
 11%|█████████▎                                                                          | 1/9 [00:03<00:24,  3.09s/it]C:\Users\zhav1k\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an erro

 33%|████████████████████████████                                                        | 3/9 [00:10<00:21,  3.64s/it]C:\Users\zhav1k\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zhav1k\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
 44%|█████████████████████████████████████▎                                              | 4/9 [00:18<00:22,  4.57s/it]C:\Users\zhav1k\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an erro

 67%|████████████████████████████████████████████████████████                            | 6/9 [00:28<00:14,  4.74s/it]C:\Users\zhav1k\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zhav1k\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
 78%|█████████████████████████████████████████████████████████████████▎                  | 7/9 [00:36<00:10,  5.24s/it]C:\Users\zhav1k\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an erro

  0%|                                                                                            | 0/9 [00:00<?, ?it/s]C:\Users\zhav1k\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zhav1k\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
 11%|█████████▎                                                                          | 1/9 [00:01<00:15,  1.88s/it]C:\Users\zhav1k\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an erro


0.5273689334843417


In [60]:
score, params, lst_train, lst_test, all_params = hyperparameter_search(5,param_dist_lr, years, df1, burning_period, evaluate_model_logistic)

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:11<00:00,  1.31s/it]



0.5276253434541142


In [24]:
score, params = hyperparameter_search(2,param_dist_rf, years, df1, burning_period, evaluate_model_rf)

[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:   36.4s
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 650 out of 650 | elapsed:  1.4min finished
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    1.2s
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed:    2.9s
[Parallel(n_jobs=4)]: Done 650 out of 650 | elapsed:    3.0s finished
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   17.8s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 650 out of 650 | elapsed:  3.2min finished
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    2.7s
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed:    6.1s
[Parallel(n_jobs=4)]: Done 650 out of 650 | elapse

Wall time: 1h 24min 23s


[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed:    3.1s
[Parallel(n_jobs=4)]: Done 650 out of 650 | elapsed:    3.1s finished


In [ ]:
%%time
score, params = hyperparameter_search(2,param_dist_svc, years, df1, burning_period, evaluate_model_svm)

0.5262791532396613

In [ ]:
best_score = 0
for i in range(1, 20): #optimization t
    params = {k: np.random.choice(v) for k, v in param_dist_lgbm.items()}
    score_test = []
    gc.collect()
    for num_year in range(1, len(years) - ):
        up_to = pd.to_datetime('2009-01-01') + timedelta(days = num_year*365)
        train_index = df1[df1['Date_datetime'] <= up_to].index
        #train = market_train[market_train['date_dt'] <= up_to]
        validation_index = df1[(df1['Date_datetime'] > (up_to + burning_period)) & (df1['Date_datetime']
                                                                                    <= up_to +timedelta(days = (num_year) * 365))].index
        
        
        
#         print('train index: ', df1.loc[train_index, fcol].shape)
#         print('val index: ', df1.loc[validation_index, fcol].shape)
        score_test.append(evaluate_model_lgbm(df1.loc[train_index, fcol], df1.loc[train_index, 'target'], 
                                   df1.loc[validation_index, fcol], df1.loc[validation_index, 'target'],
                                   params))
        
        del up_to, train_index, validation_index; gc.collect()
    if sum(score_test)/float(len(score_test)) > best_score or best_score == 0:
        best_score = sum(score_test)/float(len(score_test))
        best_params = params
        print("Best evaluation score: ", best_score)

In [ ]:
best_score

In [ ]:
best_params

In [18]:
def hyperparameter_search(rounds, grid, years, df1, burning_period, eval_model):
    best_score = 0
    best_params = []
    score_test = []
    score_train = []
    param_distr = []
    for i in tqdm_notebook(range(1, rounds)): #optimization t
        params = {k: np.random.choice(v) for k, v in grid.items()}

        gc.collect()
        param_distr.append(params)
        for num_year in tqdm(range(1, len(years) - 1)):
            up_to = pd.to_datetime('2009-01-01') + timedelta(days = num_year*365)
            train_index = df1[df1['Date_datetime'] <= up_to].index
            #train = market_train[market_train['date_dt'] <= up_to]
            validation_index = df1[(df1['Date_datetime'] > (up_to + burning_period)) & (df1['Date_datetime']
                                                                                    <= up_to +timedelta(days = (num_year) * 365))].index
        
        
        
#         print('train index: ', df1.loc[train_index, fcol].shape)
#         print('val index: ', df1.loc[validation_index, fcol].shape)
            trn, tst = eval_model(df1.loc[train_index, fcol], df1.loc[train_index, 'target'], 
                                   df1.loc[validation_index, fcol], df1.loc[validation_index, 'target'],
                                   params)
            score_train.append(trn)
            score_test.append(tst)
            del up_to, train_index, validation_index; gc.collect()
        if sum(score_test)/float(len(score_test)) > best_score or best_score == 0:
            best_score = sum(score_test)/float(len(score_test))
            best_params = params
    print(best_score)
    return(best_score, best_params, np.array(score_train).reshape(-1, 9), np.array(score_test).reshape(-1,9), param_distr)
